In [ ]:
%matplotlib inline
import pylab as pl
import numpy as np
from scipy import stats
import scipy.io.wavfile as wavf
import os
import math
import sys

def readWavFile(name):
    frequencySample, signal = wavf.read(name)
    return frequencySample, signal

def preprocessSignal(signal):
    signal = makeMono(signal)
    signal = removeSilence(signal)
    return signal

def makeMono(signal):
    finalSignal = []
    for s in signal:
        if isNumber(s):
            finalSignal.append(s)
        else:
            stereoMean = np.int16((s[0] + s[1]) / 2)
            finalSignal.append(stereoMean)
            
    return np.array(finalSignal)

def isNumber(value):
    return type(value) is np.int16

def removeSilence(signal):
    finalSignal = []
    threshold = 400
    for s in signal:
        if abs(s) > threshold:
            finalSignal.append(s)
    
    return np.array(finalSignal)

def top_peaks(signal, freqSample, num = 10):
    freqs = pl.fftfreq(len(signal))
    freqs = freqs * freqSample
    for i in range(len(freqs)):
        if freqs[i]<50:
            signal[i]=0
        elif freqs[i]>250:
            signal[i]=0            
    peaks = []            
    for i in range(num):
        k = pl.argmax(signal)
        freq = freqs[k]
        peaks.append(freq)
        signal[k]=0        
    return peaks

def check_voice(wav_name):
    freqSample, signal = readWavFile('train/' + wav_name)
    signal = preprocessSignal(signal)
    peaks = top_peaks(signal, freqSample, 100)
    #print(peaks)
    avr = np.mean(peaks)
    print(stats.mode(peaks))
    if avr > 160:
        print(wav_name + ': ---K---: ' + str(avr))
    elif avr > 50 and avr < 130:
        print(wav_name + ': ---M---: ' + str(avr))
    else:
        print(wav_name + ': ---?---: ' + str(avr))

if __name__ == '__main__':
#     frequencySample, signal = readWavFile(sys.argv[0]) # final version
#    freqSample, signal = readWavFile('train/005_M.wav') # test version
    for wav_file in os.listdir('train'):
        #print(check_voice(wav_file))
        try:
            analyze_wav(wav_file)
        except:
            pass

In [ ]:
from aubio import source, pitch

def analyze_wav(wav_file):
    wav_file = 'train/' + wav_file
    win_s = 4096
    hop_s = 512
    samplerate, signal = wavf.read(wav_file)

    s = source(wav_file, samplerate, hop_s)
    samplerate = s.samplerate
    pitch_o = pitch("yin", win_s, hop_s, samplerate)
    pitches = []
    while True:
        samples, read = s()
        pitch_sample = pitch_o(samples)[0]
        if pitch_sample > 0 and pitch_sample < 280:
            pitches += [pitch_sample]
        if read < hop_s: break
    avr = np.mean(pitches)
    q25 = np.percentile(pitches, 25)
    q75 = np.percentile(pitches, 75)
    iqr = q75 - q25
    if avr >= 140:
        gender = 'K'
    else:
        gender = 'M'
        
    print(wav_file + ' ~~~> ' + gender + ' ~~~> MEAN: ' + str(avr) + ' ~~~> IQR: ' + str(iqr) )
    #return np.mean(pitches)
